In [105]:
%pylab tk
%load_ext cythonmagic

import importlib
import numpy as np
import os
from glob import glob
import json
from time import time
from skimage import io, color
from scipy.spatial import Delaunay
from matplotlib.widgets import Cursor, Button

import helpers.features as fh
import helpers.display as dh
import helpers.geometry as gh
reload(fh)
reload(dh)
reload(gh)

BASE_PATH = os.getcwd()
print("Current base path: {0}".format(BASE_PATH))
DATA_PATH = BASE_PATH + '/Daten/2D/Talus_dorsal_cut_2/'
TH_PATH = BASE_PATH + '/Daten/2D/Talus_dorsal_cut_and_thresholded_and_clustered/'
TO_PATH = BASE_PATH + '/Daten/2D/Talus_dorsal_mesh/'

ALPHA = 25

def do_triangulation(bone_pixels):
    indices_of_bone_pixels = np.nonzero(bone_pixels)
    indices_of_bone_pixels = np.vstack(indices_of_bone_pixels).transpose()
    
    #tb = time()
    triangulation = Delaunay(indices_of_bone_pixels)
    #print(time() - tb)
    to_delete = alpha_shape_unique(triangulation.points[triangulation.simplices], alpha=ALPHA)
    triangulation.simplices = np.delete(triangulation.simplices, to_delete, axis=0)
    #print(time() - tb)
    
    return triangulation


Populating the interactive namespace from numpy and matplotlib
The cythonmagic extension is already loaded. To reload it, use:
  %reload_ext cythonmagic
Current base path: /home/stefan/Dropbox/Masterarbeit


In [102]:
%%cython

cimport numpy as np
import numpy as np
cimport cython
from libc.math cimport sqrt

@cython.boundscheck(False)
@cython.wraparound(False)
def alpha_shape_unique(np.ndarray[np.float64_t, ndim=3] simplices, float alpha=25):
    cdef np.ndarray[np.float64_t, ndim=1] pa
    cdef np.ndarray[np.float64_t, ndim=1] pb
    cdef np.ndarray[np.float64_t, ndim=1] pc
    cdef float a, b, c, s, area, circum_r
    cdef list triangles_to_delete
    cdef unsigned int index
    
    triangles_to_delete = []
    index = 0
    length = len(simplices)
    while index < length:
        pa = simplices[index, 0, :]
        pb = simplices[index, 1, :]
        pc = simplices[index, 2, :]
        # Lengths of sides of triangle
        a = sqrt((pa[0]-pb[0])**2 + (pa[1]-pb[1])**2)
        b = sqrt((pb[0]-pc[0])**2 + (pb[1]-pc[1])**2)
        c = sqrt((pc[0]-pa[0])**2 + (pc[1]-pa[1])**2)
        # Semiperimeter of triangle
        s = (a + b + c)/2.0
        # Area of triangle by Heron's formula
        area = sqrt(s*(s-a)*(s-b)*(s-c))
        circum_r = a*b*c/(4.0*area)
        # Here's the radius filter.
        if circum_r > alpha:
            triangles_to_delete.append(index)
        index += 1
    return triangles_to_delete

In [106]:
all_files = glob(TH_PATH + '*')
loaded = []

def display_triangulation(index):
    def save_triangulation(bone_pixels):
        tri = do_triangulation(bone_pixels)
        io.imsave(edited_path, bone_pixels)
        np.savez(tri_path, points=tri.points, triangles=tri.simplices)
    def rst(ev):
        print("Reset")
        plt.close(fig)
        display_triangulation(index)
    def nxt(ev):
        print("Next")
        plt.close(fig)
        save_triangulation(bone_pixels)
        display_triangulation(index+1)
    def prv(ev):
        print("Previous")
        plt.close(fig)
        save_triangulation(bone_pixels)
        display_triangulation(index-1)
    
    filename = all_files[index]
    basename = os.path.splitext(os.path.basename(filename))[0]
    edited_path = TO_PATH + basename + '.png'
    tri_path = TO_PATH + basename + '.npz'
    rgb_image_path = glob(DATA_PATH + basename + '*')[0]
    
    print("Editing {0}".format(basename))
    
    rgb_image = io.imread(rgb_image_path)
    bone_pixels = io.imread(edited_path) if (os.path.exists(edited_path)) else io.imread(filename)
    
    fig, axes = dh.triangulation(plt, rgb_image, bone_pixels, do_triangulation)
    #fig.suptitle(basename)
    
    axprev = fig.add_axes([0.02, 0.02, 0.25, 0.04])
    axreset = fig.add_axes([0.375, 0.02, 0.25, 0.04])
    axnext = fig.add_axes([0.73, 0.02, 0.25, 0.04])
    bnext = Button(axnext, 'Next')
    bnext.on_clicked(nxt)
    breset = Button(axreset, 'Reset')
    breset.on_clicked(rst)
    bprev = Button(axprev, 'Previous')
    bprev.on_clicked(prv)
    
    fig._bnext = bnext
    fig._breset = breset
    fig._bprev = bprev
    
    #fig_manager = plt.get_current_fig_manager()
    #fig_manager.window.showMaximized()
    
display_triangulation(0)

Editing GÖ_3_2011_15_R
Previous
Editing GÖ_p_2013_188L
Previous
Editing GÖ_i_2008_7_L